<a href="https://colab.research.google.com/github/nerthiga/Introduction_to_SQL_Udacity/blob/main/Quiz4_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
SELECT AVG(standard_qty)AS st_avg, AVG(gloss_qty) AS gloss_avg, AVG(poster_qty) AS poster_avg
FROM orders
WHERE DATE_TRUNC('month', occurred_at)=
(SELECT DATE_TRUNC('month', MIN(occurred_at)) AS      min_month
FROM orders);

In [ ]:
SELECT t3.name, t2.region, t2.total_amt
FROM
  (SELECT region, MAX(total) AS total_amt
  FROM 
     (SELECT r.name AS region, s.name AS name,           SUM(o.total_amt_usd) AS total
      FROM orders o
      JOIN accounts a
      ON o.account_id = a.id
      JOIN sales_reps s
      ON a.sales_rep_id = s.id
      JOIN region r
      On r.id=s.region_id
      GROUP BY 1,2) t1
   GROUP BY 1)t2
JOIN
(SELECT r.name AS region, s.name AS name, SUM(o.total_amt_usd) AS total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
JOIN region r
On r.id=s.region_id
GROUP BY 1,2
ORDER BY 3 DESC) t3
ON t2.region=t3.region AND t2.total_amt=t3.total;

In [ ]:
SELECT r.name, COUNT(o.total) as total_count
FROM orders o
      JOIN accounts a
      ON o.account_id = a.id
      JOIN sales_reps s
      ON a.sales_rep_id = s.id
      JOIN region r
      On r.id=s.region_id
      GROUP BY 1
HAVING SUM(o.total_amt_usd)=      
(SELECT MAX(total)
 FROM
   (SELECT r.name AS region,               SUM(o.total_amt_usd) AS total
      FROM orders o
      JOIN accounts a
      ON o.account_id = a.id
      JOIN sales_reps s
      ON a.sales_rep_id = s.id
      JOIN region r
      On r.id=s.region_id
      GROUP BY 1)sub);


In [ ]:
SELECT COUNT(*)
FROM
(SELECT a.name
FROM orders o
JOIN accounts a
ON a.id= o.account_id
GROUP BY 1
HAVING SUM(o.total)> (SELECT total_amt
    FROM                  
   (SELECT a.name, SUM(o.standard_qty) total_orders, SUM(o.total) AS total_amt
FROM orders o
JOIN accounts a
ON a.id=o.account_id
GROUP BY 1
ORDER BY 2 DESC    
LIMIT 1)sub))sub_two;

In [ ]:
SELECT w.channel, a.name, COUNT(*)
FROM accounts a
JOIN web_events w
ON a.id = w.account_id AND a.id= 
(SELECT id 
FROM       
(SELECT a.id as id, a.name as act_name, SUM(o.total_amt_usd) as total_spent
FROM orders o
JOIN accounts a
ON a.id = o.account_id                      
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 1) query_one)
GROUP BY 1,2
ORDER BY 3 DESC;

In [ ]:
SELECT AVG(total_spent)                      
FROM (SELECT a.name, SUM(o.total_amt_usd) AS total_spent
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10)sub_query ;

In [ ]:
SELECT AVG(avg)
FROM                      
(SELECT o.account_id, AVG(o.total_amt_usd) as avg
FROM orders o
GROUP BY 1
HAVING AVG(total_amt_usd)> SELECT AVG(o.total_amt_usd) as tot_avg
FROM orders o))sub_query;

In [ ]:
WITH t1 AS (SELECT r.name AS region, s.name AS    name, SUM(o.total_amt_usd) AS total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
JOIN region r
On r.id=s.region_id
GROUP BY 1,2
ORDER BY 3 DESC),

t2 AS (SELECT region, MAX(total) AS total_amt
       FROM t1
       GROUP BY 1)
       
SELECT t1.name, t2.region, t2.total_amt
FROM t1
JOIN t2
ON t2.region=t1.region AND t2.total_amt=t1.total;


In [ ]:
WITH t1 AS (SELECT r.name AS region,                     SUM(o.total_amt_usd) AS total
      FROM orders o
      JOIN accounts a
      ON o.account_id = a.id
      JOIN sales_reps s
      ON a.sales_rep_id = s.id
      JOIN region r
      On r.id=s.region_id
      GROUP BY 1),
      
t2 AS (SELECT MAX(total)
       FROM t1)
       
 SELECT r.name, COUNT(o.total) as total_count
FROM orders o
      JOIN accounts a
      ON o.account_id = a.id
      JOIN sales_reps s
      ON a.sales_rep_id = s.id
      JOIN region r
      On r.id=s.region_id
      GROUP BY 1
HAVING SUM(o.total_amt_usd) = (SELECT * FROM t2); 


In [ ]:
WITH t1 AS (SELECT a.name, SUM(o.standard_qty) total_orders, SUM(o.total) AS total_amt
FROM orders o
JOIN accounts a
ON a.id=o.account_id
GROUP BY 1
ORDER BY 2 DESC    
LIMIT 1),

t3 AS (SELECT a.name
FROM orders o
JOIN accounts a
ON a.id= o.account_id
GROUP BY 1
HAVING SUM(o.total)>(SELECT total_amt FROM t1))

SELECT COUNT(*)
FROM t3;
                     